In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import callbacks
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Dropout
import matplotlib.pyplot as plt
from sklearn import metrics
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

import keras
from keras import *
from keras.layers import *
from pythainlp import word_tokenize
from pythainlp.word_vector import *

from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('main_suicidal_data.csv').drop(columns='Unnamed: 0')

In [3]:
wModel = get_model()
thai2dict = {}
for word in wModel.index_to_key:
    thai2dict[word] = wModel[word]
thai2vec = pd.DataFrame.from_dict(thai2dict,orient='index')
thVocab = thai2vec.index.to_list()

In [4]:
ll = len(thai2vec)
for vidx in range(ll):
    if vidx % 100 == 0:
        print('\r' + str(vidx),end='')
    aa = thai2vec.iloc[[vidx]]
    ab = aa.values.tolist()
    if vidx == 0:
        vect = ab
    else:
        vect = np.vstack((vect,ab))

print("\n", vect.shape)

51300
 (51358, 300)


In [5]:
def tokenWord(wordTarget):
    wordToken = word_tokenize(wordTarget, engine='attacut')
    return wordToken

def convWord(cw):
    cWord = cw
    for ti in range(len(cWord)):
        if cWord[ti] == ' ':
            cWord[ti] = ''
        elif cWord[ti] not in thVocab:
            cWord[ti] = ''
    return cWord

def token2index(t2idx):
    w2index = []
    for wi in range(len(t2idx)):
        if t2idx[wi] in thVocab:
            w2index.append(thVocab.index(t2idx[wi]))
    return np.array(w2index)

def findMaxArray(fma):
    maxlen = 0
    for mi in range(len(fma)):
        nA = len(fma[mi])
        if nA > maxlen:
            maxlen = nA
    return maxlen

def fill0in(f0i):
    fMax = findMaxArray(f0i)
    for fi, ax in enumerate(f0i):
        if len(ax) < fMax:
            f0i[fi] = np.hstack((ax , np.zeros(fMax-len(ax))))
        f0i1 = np.array(f0i)
    return f0i1

def prepare2train(ipt):
    pre2t = []
    for pidx in range(len(ipt)):
        wp1 = ipt[pidx]
        pre2t.append(token2index(convWord(tokenWord(wp1))))
    return pre2t

In [6]:
X = df['Tweet']
X_arr = X.to_list()
y = df['Label (Specialist)']

In [7]:
le = LabelEncoder()
le.fit(y)

LabelEncoder()

In [8]:
X = prepare2train(X_arr)
y = le.transform(y)

In [9]:
y = np.array(y)
X = fill0in(X)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# LSTM

In [28]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vect.shape[0],vect.shape[1],name='embed'))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(6, activation="sigmoid"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed (Embedding)           (None, None, 300)         15407400  
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              186880    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 774       
                                                                 
Total params: 15,595,054
Trainable params: 15,595,054
Non-trainable params: 0
_________________________________________________________________


In [29]:
from tensorflow.keras.optimizers import SGD
opt = SGD(lr=0.1)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [30]:
model.get_layer('embed').set_weights([vect])
model.get_layer('embed').trainable = True

In [14]:
class myCallback(callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      self.model.stop_training = True
      print("\nCallback Accuracy มากกว่า 95%!")
callbacks = myCallback()

In [15]:
model.fit(X_train,y_train,epochs=30,batch_size=16,callbacks=[callbacks])

2022-12-29 10:25:36.614051: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/30


2022-12-29 10:25:38.519101: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 10:25:38.902686: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 10:25:38.920064: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 10:25:40.186123: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 10:25:40.211825: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


289/289 [==============================] - 27s 78ms/step - loss: 1.3645 - accuracy: 0.5451
Epoch 2/30
289/289 [==============================] - 22s 75ms/step - loss: 1.2102 - accuracy: 0.5752
Epoch 3/30
289/289 [==============================] - 22s 74ms/step - loss: 1.1116 - accuracy: 0.6178
Epoch 4/30
289/289 [==============================] - 22s 77ms/step - loss: 1.0410 - accuracy: 0.6377
Epoch 5/30
289/289 [==============================] - 21s 74ms/step - loss: 0.9813 - accuracy: 0.6602
Epoch 6/30
289/289 [==============================] - 22s 75ms/step - loss: 0.9504 - accuracy: 0.6780
Epoch 7/30
289/289 [==============================] - 21s 73ms/step - loss: 0.9067 - accuracy: 0.6877
Epoch 8/30
289/289 [==============================] - 21s 73ms/step - loss: 0.8702 - accuracy: 0.7027
Epoch 9/30
289/289 [==============================] - 21s 72ms/step - loss: 0.8332 - accuracy: 0.7124
Epoch 10/30
289/289 [==============================] - 21s 73ms/step - loss: 0.8039 - accurac

In [16]:
model_acc = model.evaluate(X_test,y_test)

2022-12-29 10:36:23.824916: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 10:36:23.990009: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 10:36:24.003935: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - 2s 64ms/step - loss: 1.2103 - accuracy: 0.6790


In [17]:
y_pred = np.argmax(model.predict(X_test),axis=1)

2022-12-29 10:36:25.701611: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 10:36:25.827207: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 10:36:25.839932: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.86      0.83       264
           1       0.44      0.47      0.45        53
           2       0.53      0.42      0.47        24
           3       0.42      0.44      0.43        39
           4       0.55      0.50      0.52        66
           5       0.69      0.53      0.60        68

    accuracy                           0.68       514
   macro avg       0.57      0.54      0.55       514
weighted avg       0.67      0.68      0.67       514



# Try it with Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier() 
clf.fit(X_train, y_train)
y_pred_rf_not_tuning = clf.predict(X_test)

In [20]:
print(classification_report(y_test, y_pred_rf_not_tuning))

              precision    recall  f1-score   support

           0       0.61      0.95      0.74       264
           1       0.76      0.30      0.43        53
           2       0.58      0.29      0.39        24
           3       0.43      0.15      0.23        39
           4       0.64      0.32      0.42        66
           5       0.76      0.24      0.36        68

    accuracy                           0.62       514
   macro avg       0.63      0.38      0.43       514
weighted avg       0.63      0.62      0.56       514



# Tuning Param

In [21]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in range(200,2000,200)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [22]:
rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=0, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

/Users/yo/miniforge3/envs/mlp/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/yo/miniforge3/envs/mlp/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/yo/miniforge3/envs/mlp/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this p

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800]},
                   random_state=42)

In [23]:
print(rf_random.best_estimator_)

RandomForestClassifier(bootstrap=False, max_depth=30, max_features='auto',
                       n_estimators=200)


In [26]:
rf_tuned = RandomForestClassifier(bootstrap=False, max_depth=30, max_features='auto',
                       n_estimators=200)
rf_tuned.fit(X_train, y_train)
y_pred_rf_tuned = rf_tuned.predict(X_test)

In [27]:
print(classification_report(y_test, y_pred_rf_tuned))

              precision    recall  f1-score   support

           0       0.61      0.96      0.75       264
           1       0.73      0.30      0.43        53
           2       0.58      0.29      0.39        24
           3       0.44      0.18      0.25        39
           4       0.65      0.30      0.41        66
           5       0.80      0.24      0.36        68

    accuracy                           0.62       514
   macro avg       0.63      0.38      0.43       514
weighted avg       0.64      0.62      0.57       514

